# Generador de instancias
En este notebook está el código para generar los sets de instancias que se usan para experimentar.
- Estas instancias van a ser guardadas en la carpeta __instancias__.
- Cada set estará en su propia carpeta y tendrá un archivo _indice.csv_ que contendrá información sobre las instancias.

In [7]:
import random, math
import pandas as pd

In [8]:
def save_instance(dataset, instance_name, S, R):
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        print(len(S), R, file=f)
        for s in S:
            for n in s:
                print(n, file=f, end=" ")

def save_index(dataset, instances):
    with open(F"instancias/{dataset}/instances.txt", "w") as f:
        for instance in instances: 
            print(instance, file=f)

## Dataset 1
Instancias de mejor caso de backtracking, están descriptas en el informe en más detalle.


In [9]:
#mejor caso es O(n), es el mejor caso porque rechaza todo elemento menos el ultimo. Entonces termina recorriendo 
#linealmete S
filas_indice = []
for i in range(1, 1001):
    n = i*12 
    R = i*12
    S = [[R+10, n+i] for i in range(0, n)]
    S[-1] = [R, R] #esta linea accede al ultimo elemento del array
    save_instance("mejor-caso-BT", F"BT-MC-{n}", S, R)
    filas_indice.append(["mejor-caso-BT", F"BT-MC-{n}", n, R, F"instancias/mejor-caso-BT/BT-MC-{n}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/mejor-caso-BT/indice.csv", index=False, header=True)

## Dataset 2
Instancias de peor caso de backtracking-O, están descriptas en el informe en más detalle.


In [10]:
#peor caso es O(2^n) cuando solo es la poda de opt, recorre todo el arbol al
#no encontrar soluciones validas
filas_indice = []
for n in range(1, 31):
    R = random.randint(1, 1000)
    S = [[R,R] for i in range(0, n)]
    save_instance("peor-caso-BT-O", F"BT-O-PC-{n}", S, R)
    filas_indice.append(["peor-caso-BT-O", F"BT-O-PC-{n}", n, R, F"instancias/peor-caso-BT-O/BT-O-PC-{n}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/peor-caso-BT-O/indice.csv", index=False, header=True)

In [11]:
#peor caso es O(2^n) cuando solo es la poda de fact, recorre todo el arbol al
#no encontrar soluciones invalidas
filas_indice = []
for n in range(1, 31):
    R = random.randint(n, 1000)
    S = [[1,R] for i in range(0, n)]
    save_instance("peor-caso-BT-F", F"BT-F-PC-{n}", S, R)
    filas_indice.append(["peor-caso-BT-F", F"BT-F-PC-{n}", n, R, F"instancias/peor-caso-BT-F/BT-F-PC-{n}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/peor-caso-BT-F/indice.csv", index=False, header=True)

# INSTANCIA 3
soluciones largas (lion)

In [12]:
filas_indice = []
for n in range(5,55):
    R = random.randint(n,1000) 
    S =[[random.randint(1,2*math.floor(R/n)), #la idea es que los pesos sean como mucho que n/2 items llenen el tubo
         random.randint(R - math.floor(R/10), R)] for k in range(0,n)] #y las resistencias tiendan más a soportar pilas
    save_instance("sol-largas-L", F"sol-largas-L-{n}", S, R)
    filas_indice.append(["sol-largas-L", F"sol-largas-L-{n}", n, R, F"instancias/sol-largas-L/sol-largas-L-{n}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/sol-largas-L/indice.csv", index=False, header=True)

# INSTANCIA 4
soluciones cortas (lion)

In [13]:
filas_indice = []
for n in range(5,55):
    R = random.randint(n, 1000)
    S = [[random.randint(2*math.floor(R/n),R - math.floor(R/4)), #en el mejor caso las soluciones son de n/2 y en el peor son de 1
          random.randint(2*math.floor(R/n),R)] for k in range(0,n)] #la idea es que nunca haya mucha toleracia a pilas, pero que existan
    save_instance("sol-cortas-L", F"sol-cortas-L-{n}", S, R)
    filas_indice.append(["sol-cortas-L", F"sol-cortas-L-{n}", n, R, F"instancias/sol-cortas-L/sol-cortas-L-{n}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/sol-cortas-L/indice.csv", index=False, header=True)

# INSTANCIA 5
mejor caso DP posible

In [18]:
filas_indice = []
for i in range(5,60):
    n = i
    R = random.randint(n,n*5) #con los pesos como son, queremos que no nos pasemos de la resistencia del tubo mucho
    S =[[random.randint(1,math.floor(R/n)), #asi hay muchas pilas de pesos similares en el mismo i
         random.randint(2*math.floor(R/n), 5*math.floor(R/n))] for k in range(0,n)] #para que se quiebren bastante, y no haya soluciones muy grandes que usen varios espacios 
    save_instance("mejor-DP", F"mejor-DP-{n}", S, R)
    filas_indice.append(["mejor-DP", F"mejor-DP-{n}", n, R, F"instancias/mejor-DP/mejor-DP-{n}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/mejor-DP/indice.csv", index=False, header=True)    

# instancia 6 dp generales


In [17]:
filas_indice = []
for n in range(1000, 8000, 500):
       for R in range(1000, 8000, 500):
        S = [[random.randint(2*math.floor(R/n),R - math.floor(R/4)), #en el mejor caso las soluciones son de n/2 y en el peor son de 1
              random.randint(2*math.floor(R/n),R)] for k in range(0,n)]
        save_instance( "dinamica", F"DP-{n}-{R}", S, R)
        filas_indice.append(["dinamica", F"DP-{n}-{R}", n, R, F"instancias/dinamica/DP-{n}-{R}.txt"])  
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "R", "archivo"]).to_csv("instancias/dinamica/indice.csv", index=False, header=True)        
         
                